In [ ]:
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
from scipy import signal



# Obrazki
if not os.path.exists("jet.png") :
    !wget --no-check-certificate https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/jet.png 
if not os.path.exists("kw.png") :
    !wget --no-check-certificate https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/kw.png 
if not os.path.exists("moon.png") :
    !wget --no-check-certificate https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/moon.png 
if not os.path.exists("lenaSzum.png") :
    !wget --no-check-certificate https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/lenaSzum.png 
if not os.path.exists("lena.png") :
    !wget --no-check-certificate https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/lena.png 
if not os.path.exists("plansza.png") :
    !wget --no-check-certificate https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/plansza.png


plt.rcParams["figure.figsize"] = (8,8)

plansza = cv2.imread('plansza.png')
plansza = cv2.cvtColor(plansza,cv2.COLOR_BGR2GRAY)



def conv(img,size,kernel=None):
  if kernel is None:
    kernel = np.ones((size,size),np.float32)/(size**2)
  else:
    size=len(kernel)
  dst = cv2.filter2D(img,-1,kernel)
  print("Size of window is:",size)
  fig,axs = plt.subplots(1,3)
  axs[0].imshow(img,'gray')
  axs[0].set_title('original')
  axs[1].imshow(dst,'gray')
  axs[1].set_title('averaging')
  img_diff=abs(img-dst)
  axs[2].imshow(img_diff,'gray')
  axs[2].set_title('difference')
  plt.show()

  return dst

# # ----dolnoprzepustowa----
pl3 = conv(plansza,3)
pl5 = conv(plansza,5)
pl9 = conv(plansza,9)
pl15 = conv(plansza,15)
pl35 = conv(plansza,35)


fig,axs = plt.subplots(1,5)
fig.suptitle("Comparison due to the size of the mask ")
axs[0].imshow(pl3,'gray')
axs[0].set_title('size=3')
axs[1].imshow(pl5,'gray')
axs[1].set_title('size=5')
axs[2].imshow(pl9,'gray')
axs[2].set_title('size=9')
axs[3].imshow(pl15,'gray')
axs[3].set_title('size=15')
axs[4].imshow(pl35,'gray')
axs[4].set_title('size=35')
plt.show()


lena = cv2.imread('lena.png')
lena = cv2.cvtColor(lena,cv2.COLOR_BGR2GRAY)
dst=conv(lena,3)
dst=conv(lena,5)
dst=conv(lena,9)
dst=conv(lena,15)
dst=conv(lena,35)

# inna maska


print("Maska dobrana do zdjecia")
M=np.array([[1,2,1],[2,4,2],[1,2,1]])
M = M/sum(sum(M))
dst=conv(lena,3,M)
dst=conv(lena,5,M)
dst=conv(lena,9,M)
dst=conv(lena,15,M)
dst=conv(lena,35,M)




# # maska gaussa


def fgaussian(size, sigma):
     m = n = size
     h, k = m//2, n//2
     x, y = np.mgrid[-h:h+1, -k:k+1]
     g = np.exp(-(x**2 + y**2)/(2*sigma**2))
     return g /g.sum() 
    
    
def mesh(fun, size):
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    

    X = np.arange(-size//2, size//2, 1)
    Y = np.arange(-size//2, size//2, 1)
    X, Y = np.meshgrid(X, Y)
    Z = fun
    
    ax.plot_surface(X, Y, Z)
    
    plt.show()


kernel=fgaussian(5,0.5)
mesh(kernel,5)
dst=conv(lena,5,kernel)
blur = cv2.GaussianBlur(lena,(5,5),0.5)
fig,axs = plt.subplots(1,2)
axs[0].imshow(dst,'gray')
axs[0].set_title('fgausian ')
axs[1].imshow(blur,'gray')
axs[1].set_title('GaussianBlur')
plt.show()

kernel=fgaussian(5,0.1)
mesh(kernel,5)
dst=conv(lena,5,kernel)
kernel=fgaussian(5,0.99)
mesh(kernel,5)
dst=conv(lena,5,kernel)




# ---filtry nieliniowe-----

def median(img,size):
  print("Size of window is:",size)
  median = cv2.medianBlur(img,size)
  fig,axs = plt.subplots(1,3)
  axs[0].imshow(img,'gray')
  # axs[0].imshow(np.real(img),'gray')
  axs[0].set_title('original')
  axs[1].imshow(median,'gray')
  
  axs[1].set_title('median')
  img_diff=abs(img-median)
  axs[2].imshow(img_diff,'gray')
  axs[2].set_title('difference')
  plt.show()

lenaSzum = cv2.imread('lenaSzum.png')
lenaSzum = cv2.cvtColor(lenaSzum,cv2.COLOR_BGR2GRAY)

print('median for lenaSzum')
dst = conv(lenaSzum,3)
med =median(lenaSzum,3)

print('median for lena')
dst = conv(lena,3)
med =median(lena,3)



lenaConv=lena
for i in range(10):     #first time was before the loop
  lenaConv = cv2.medianBlur(lenaConv,5)
fig,axs = plt.subplots(1,3)
axs[0].imshow(lena,'gray')
axs[0].set_title('original')
axs[1].imshow(lenaConv,'gray')
axs[1].set_title('multiple median filtration')
img_diff=abs(lena-lenaConv)
axs[2].imshow(img_diff,'gray')
axs[2].set_title('difference')
plt.show()



# ----gornoprzepustowe

def lapl(img,M):
  dst = (cv2.filter2D(img,-1,M)).astype('uint8')
  dstScal = dst+128
  dstAbs=abs(dst)
  fig,axs = plt.subplots(1,3) 
  axs[0].imshow(img,'gray')
  axs[0].set_title('original')
  axs[1].imshow(dstScal,'gray')
  axs[1].set_title('skalowanie')
  axs[2].imshow(dstAbs,'gray')
  axs[2].set_title('wartosc bezwzgledna')
  plt.show()

  return dst

def sharp(img,dst):
  fig,axs = plt.subplots(1,3)
  img=img.astype('uint8')
  
  axs[0].imshow(img,'gray')
  axs[0].set_title('original')
  suma=img+dst
  axs[1].imshow(suma,'gray')
  axs[1].set_title('suma')
  diff = img-dst
  axs[2].imshow(diff,'gray')
  axs[2].set_title('roznica')
  plt.show()


moon = cv2.imread('moon.png')
moon = cv2.cvtColor(moon,cv2.COLOR_BGR2GRAY)

M=np.array([[0,1,0],[1,-4,1],[0,1,1]])
M = M/9
dst=lapl(moon,M)
sharp(moon,dst)




# -----gradienty-----

R1=np.array([[0,0,0],[-1,0,0],[0,1,0]])
R2=np.array([[0,0,0],[0,0,-1],[0,1,0]])

P1=np.array([[-1,0,1],[-1,0,1],[-1,0,1]])
P2=np.array([[-1,-1,-1],[0,0,0],[1,1,1]])

S1=np.array([[-1,0,1],[-2,0,2],[-1,0,1]])
S2=np.array([[-1,-2,-1],[0,0,0],[1,2,1]])



kw = cv2.imread('kw.png')
kw = cv2.cvtColor(kw,cv2.COLOR_BGR2GRAY)

dst=lapl(kw,R1)
dst=lapl(kw,R2)
dst=lapl(kw,P1)
dst=lapl(kw,P2)
dst=lapl(kw,S1)
dst=lapl(kw,S2)

jet = cv2.imread('jet.png')
jet = cv2.cvtColor(jet,cv2.COLOR_BGR2GRAY)


jet1 = lapl(jet,S1)
jet2=lapl(jet,S2)

imgOut = np.sqrt(np.power(jet1,2) + np.power(jet2,2)).astype('uint8')
imgOut2 = abs(jet1)+abs(jet2).astype('uint8')
fig,axs = plt.subplots(1,3) 
axs[0].imshow(jet,'gray')
axs[0].set_title('original')
axs[1].imshow(imgOut,'gray')
axs[1].set_title('sqrt')
axs[2].imshow(imgOut2,'gray')
axs[2].set_title('abs')
plt.show()
